### 순차적인 데이터를 처리하는 RNN (Recurrent neural network)
- 영어 두 문장이 있다
    - I live to eat.
    - I eat to live.
- 두 문장 같은 단어로 이뤄 졌지만, live와 eat의 위치가 바뀌면서 뜻이 달라짐
- 단어의 특징만 잡아내는 일반적인 신경망 구조로는 이 변화를 인식하기 어려움
- 데이터의 순서가 주는 정보까지 인지해내는 새로운 형태의 신경망을 배움

#### RNN 개요
- 앞에서 배운 신경망 모델들은 시간에 대한 개념이 없다.
- 하지만 현실에서 접하는 거의 모든 경험은 순차적이다.
- RNN은 순차적 데이터, 시계열 데이터 의 정보를 받아 전체 내용을 학습한다
- RNN은 정해지지 않은 길이의 배열을 읽고 설명하는 신경망
    - 시계열 데이터인 주가 정보를 입력받았다고 가정 순서대로 데이터를 훑으며 정보를 파악하고 등락률이 얼마나 될지 예측할수 있다
- RNN의 출력은 순차적 데이터의 흐름을 모두 내포
- RNN은 시계열 데이터의 정보를 하나씩 입력 받을때마다 지금까지 입력받은 벡터들을 종합해 은닉 벡터를 만들어 낸다
    - 입력1,2,3...k 까지 입력 받을 때 마다 모두 압축한 은닉벡터 생성 
    - 마지막 은닉벡터$_k$는 배열 속 모든 벡터들의 내용을 압축한 벡터라고 할 수 있다
<img src="./7-1_RNN구조.png" width="70%" height="70%">

- RNN 계열 신경망들은 대표적인 시계열 데이터인 텍스트와 자연어를 처리하고 학습하는데 주로 사용
    - LSTM(long short term memory), GRU(gated recurrent unit) 등 응용 RNN 이 개발되어 언어 모델링(language modeling), 텍스트 감정 분석(text seniment analysis), 기계 번역(machine translation) 등의 분야에서 이용
- RNN을 응용한 신경망 형태의 예
    - 일대일
        - 일대일은 일반적으로 보아온 신경망이나 CNN과 같다
    <img src="./7_2_1_일대일.png" width="20%" height="20%">
    - 일대다
        - 일대다는 이미지를 보고 이미지 안의 상황을 글로 설명하는 등의 문제
    <img src="./7_2_2_일대다.png" width="70%" height="70%">
    - 다대일
        - 다대일은 감정 분석 같이 순차적인 데이터를 보고 값 하나를 내는 경우
    <img src="./7_2_3_다대일.png" width="70%" height="70%">
    - 다대다
        - 다대다는 챗봇과 기계 번역 같이 순차적인 데이터를 보고 순차적인 데이터를 출력
    <img src="./7_2_4_다대다.png" width="70%" height="70%">
    - 다대다
        - 비디오 분류 같이 매 프레임을 레이블링 할 때 사용하는 구조
    <img src="./7_2_5_다대다.png" width="70%" height="70%">

#### 영화 리뷰 감정 분석
- 데이터의 순서 정보를 학습한다는 점에서 RNN 은 CIFAR-10 같은 정적 데이터 보다는 동영상, 자연어, 주가 등 동적인 데이터를 이용할 때 성능이 극대화
- 가장 기본적인 자연어 처리 작업이라고 할 수 있는 텍스트 감정 분석
- 텍스트 형태의 데이터셋인 IMDB는 영화리뷰 5만건으로 이루어졌다
- 다수의 영어 문장으로 구성, 긍정적 리뷰 2, 부정적 1로 레이블링
- RNN 을 이용한 영화 리뷰 감정 분석
    <img src="./7_3_RNN_영화감정분석.png" width="70%" height="70%">
- 토큰나이징
    - 언어의 최소 단위인 토큰으로 나누는 것
- 워드 임베딩
    - 언어의 최소 단위인 토큰을 벡터형태로 변환 하는 작업

In [46]:
# 자연어의 전처리 과정은 개념적으로 어렵진 않지만 코딩량이 많고 버그를 만들기 쉬움
# 다행이도 토치텍스트의 전처리 도구들과 파이토치의 nn.Embedding 같은 기능을 이용하면 된다

# 라이브러리 import
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [53]:
# 하이퍼 파라미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 40
# EPOCHS = 5
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [54]:
# IMDB 데이터셋 로딩하고 텐서로 변환
# 텍스트 형태 영화 리뷰들과 그에 해당하는 레이블을 텐서로 바꿔줄 때 필요한 설정 정함
# sequential 파라미터를 이용해 데이터셋이 순차적인 테이터셋인지 명시
# 레이블 값은 단순히 클래스를 나타내는 숫자이므로 순차적인 데이터가 아님
# batch_first 파라미터로 신경망에 입력되는 텐서의 첫 번째 차원값이 batch_size가 되도록 정해줌
# lower 변수를 이용해 텍스트 데이터 속 모든 영문 알파벳이 소문자가 되도록 처리
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
 
# dataset 객체의 splits() 함수를 이용해 모델에 입력되는 데이터셋을 만듬
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

# 워드 임베딩에 필요한 단어 사전을 만듬
# min_freq 는 학습 데이터에서 최소 5번 이상 등장한 단어만 사전에 담겠다는 의미
# 그 미만으로 출현하는 단어는 'unk' (unknown) 토큰으로 대체
TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

In [55]:
# IMDB 데이터셋에선 따로 검증셋이 존재하지 않기에 학습셋을 쪼개서 사용
# batch 단위로 쪼개서 학습을 진행
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
                                (trainset, valset, testset),
                                batch_size=BATCH_SIZE, shuffle=True, repeat=False)

# 사전 속 단어들의 개수와 레이블의 수를 정해주는 변수를 만듬
vocab_size = len(TEXT.vocab)
n_classes = 2

In [56]:
print('[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스]: %d' 
     % (len(trainset), len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스]: 2


- 다층(multilayer) 형태의 RNN
<img src="./7_4_다층형태.png" width="70%" height="70%">

In [57]:
# RNN 모델 구현
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicGRU, self).__init__()
        print('Building Basic GRU model...')
        
        # 가장 먼저 정의하는 변수는 은닉 벡터들의 '층' 이라고 할수 있는 n_layers 이다
        # 아주 복잡한 모델이 아닌이상 n_layers는 2이하로 정의
        self.n_layers = n_layers
        
        # nn.Embedding() 함수는 두개를 입력 받음
        # 첫 번째 파라미터(n_vocab)는 전체 데이터셋의 모든 단어를 사전형태로 나타내었을 때 그 사전에 등재된 단어 수
        # 두 번째 파라미터(embed_dim)는 임베딩된 단어 텐서가 지나는 차원 값
        self.embed = nn.Embedding(n_vocab, embed_dim)
        
        # 은닉 벡터의 차원값과 드롭아웃을 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        
        # RNN 모델 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim, 
                          num_layers=self.n_layers, batch_first=True)
        
        # RNN이 아닌 GRU 사용이유
        # 사람도 긴 문장을 읽다보면 앞부분 맥락이 기억나지 않을때가 있듯 
        # 딥러닝 모델도 문장 뒷 부분에 다다를수록 정보가 소실됨
        # 기본적인 RNN은 입력이 길어지면 학습 도중 기울기가 너무 작아지거나 커져서 
        # 앞부분에 정보를 정확이 못담을수 있음
        # 경사도 폭발, 소실 가능
        # 결함을 보완하는 더 발전된 형태의 GRU 사용
        # GRU는 시계열 데이터 속 벡터 사이의 정보 전달량을 조절
        # 기울기를 적정하게 유지, 문장 앞부분의 정보가 끝까지 도달할 수 있도록 도와줌
        
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        # 모델에 입력되는 x는 한 배치속에 있는 모든 영화평
        # 영화평을 embed() 함수로 워드임베딩 시키면 백터의 배열, 즉 시계열 데이터로 변환
        x = self.embed(x)
        
        # 보통의 신경망이라면 이제 바로 신경망 모듈의 forward() 함수를 호출해도 되지만
        # RNN 계열 신경망은 입력 데이터 외에도 은닉벡터를 정의해 x와 함께 임력해줘야 함
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 은닉 벡터는 보통 모든 특성값이 0인 벡터로 설정
        
        # 은닉벡터 h_0과 함께 self.gru() 함수에 입력하면 은닉 벡터들이 시계열 배열 형태로 변환, 3d 텐서
        # self.gru() 함수가 반환한 텐서를 [:,-1,:]로 인덱싱 하면 
        # 배치내 모든 시계열 은닉 벡터들의 마지막 토큰들을 내포한 (batch_size, 1, hidden_dim) 모양의 텐서 추출
        # h_t가 곧 영화 리뷰 배열들을 압축한 은닉 벡터
        x, _ = self.gru(x, h_0)
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
        
        
    def _init_state(self, batch_size=1):
        # parameters() 함수는 신경망 모듈 의 가중치 정보들을 반복자 형태로 반환
        # next(self.parameters()).data는 nn.GRU 모듈의 첫번째 가중치 텐서를 추출
        # 그후 new() 함수를 호출해 모델의 가중치와 같은 모양인 (n_layers, batch_size, hidden_dim) 모양을 
        # 갖춘 텐서로 변환후 zero_() 함수를 호출해 텐서 내 모든 값을 0으로 초기화
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [58]:
# 학습 함수와 평가 함수를 구현
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        
        # label은 1 또는 2의 값 -> 0,1 로 바꾸자
        y.data.sub_(1) # 모든 값에서 1씩 빼는 함수
        
        # 매번 기울기를 새로 계산 하므로 기울기를 0으로
        optimizer.zero_grad()
        logit = model(x)
        
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum') # 오차의 합을 구함
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum() # 모델이 맞친 수
    
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * corrects / size
    return avg_loss, avg_accuracy

In [59]:
# 학습 전 모델 객체를 정의
# 은닉 벡터 차원값으론 256, 임베딩 토큰의 차원값은 128로 임의 설정
# 최적화 알고리즘 선택, 무슨 최적화 알고리즘을 사용할지 모를 때는 Adam을 사용하라는 말이 있음
# 최적화 알고리즘 비교 : https://sacko.tistory.com/42

model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

# 학습
best_val_loss = None
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print('[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f' % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적화 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir('snapshot'):
            os.makedirs('snapshot')
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

Building Basic GRU model...
[이폭: 1] 검증 오차: 0.70 | 검증 정확도:50.00
[이폭: 2] 검증 오차: 0.70 | 검증 정확도:50.00
[이폭: 3] 검증 오차: 0.61 | 검증 정확도:67.00
[이폭: 4] 검증 오차: 0.34 | 검증 정확도:85.00
[이폭: 5] 검증 오차: 0.35 | 검증 정확도:85.00
[이폭: 6] 검증 오차: 0.37 | 검증 정확도:86.00
[이폭: 7] 검증 오차: 0.40 | 검증 정확도:85.00
[이폭: 8] 검증 오차: 0.42 | 검증 정확도:86.00
[이폭: 9] 검증 오차: 0.46 | 검증 정확도:87.00
[이폭: 10] 검증 오차: 0.49 | 검증 정확도:86.00
[이폭: 11] 검증 오차: 0.50 | 검증 정확도:87.00
[이폭: 12] 검증 오차: 0.48 | 검증 정확도:86.00
[이폭: 13] 검증 오차: 0.57 | 검증 정확도:86.00
[이폭: 14] 검증 오차: 0.58 | 검증 정확도:86.00
[이폭: 15] 검증 오차: 0.56 | 검증 정확도:86.00
[이폭: 16] 검증 오차: 0.60 | 검증 정확도:86.00
[이폭: 17] 검증 오차: 0.58 | 검증 정확도:87.00
[이폭: 18] 검증 오차: 0.60 | 검증 정확도:87.00
[이폭: 19] 검증 오차: 0.58 | 검증 정확도:87.00
[이폭: 20] 검증 오차: 0.66 | 검증 정확도:86.00
[이폭: 21] 검증 오차: 0.69 | 검증 정확도:86.00
[이폭: 22] 검증 오차: 0.62 | 검증 정확도:86.00
[이폭: 23] 검증 오차: 0.63 | 검증 정확도:87.00
[이폭: 24] 검증 오차: 0.62 | 검증 정확도:87.00
[이폭: 25] 검증 오차: 0.63 | 검증 정확도:87.00
[이폭: 26] 검증 오차: 0.69 | 검증 정확도:87.00
[이폭: 27] 검증 오차: 0.75 | 검증 정확도:87.00
[이폭: 28] 

In [60]:
# 테스트셋으로 모델 성능 시험
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.34 | 테스트 정확도: 85.00


#### Seq2Seq 기계 번역
- 2010년 이후 화제가 된 머신러닝 모델이 있다.
- 언어를 다른 언어로 해석해주는 뉴럴 기계 번역(neural machine translation) 모델이다
- RNN 기반의 번역 모델인 Sequence to Sequence 모델은 기계 번역의 새로운 패러다임을 열었다고 할 정도로 뛰어난 성능을 보인다
- Seq2Seq 모델은 시퀀스를 입력받아 또다른 시퀀스를 출력
- 한마디로 다른 문장을 번역해주는 모델
- 이러한 능력을 학습하려면 병렬 말뭉치(parallel corpora) 라고 하는 원문과 번역문이 쌍을 이루는 형태의 많은 텍스트 데이터가 필요
- 이번 예제는 Seq2Seq 모델을 아주 간소화
- Seq2Seq 개요
    - 각자 다른 역할을 하는 RNN을 이어붙인 모델
    - 외국어를 한국어로 번역하는 과정
        - 외국어 문장을 읽고 의미를 이해
        - 의미를 생각하면서 한국어 단어를 한자 한자 문맥에 맞게 적어감
    - 이 두역할을 인코더와 디코더라는 두 RNN 에 부여함으로써 번역.
    <img src="./7_7_기계번역모델.png" width="70%" height="70%">
    - 인코더
        - 원문의 내용을 학습하는 RNN
        - 모든 단어를 입력받아 문장의 뜻을 내포하는 하나의 고정 크기 텐서를 만듬
        - 이렇게 압축된 텐서는 원문의 뜻과 내용을 압축하고 있다고 하여 문맥벡터(context vector)라고 함
        - 인코더는 원문속의 토큰을 차례로 입력받음
        - 원문 마지막 토큰에 해당하는 은닉 벡터는 원문의 뜻을 모두 내포하는 문맥 벡터
    - 디코더
        - 인코더와 마찬가지로 RNN 모델
        - 인코더에게서 원문 문맥 벡터를 이어받아 번역문 속의 토큰을 차례대로 예상

In [61]:
# Seq2Seq 모델 구현하기
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt

In [65]:
# 한 언어로 된 문장을 다른 언어로 번역하는 작업을 할 때 보통 단어를 문장의 최소 단위로 임베딩을 한다
# 그러나 예제에서는 글자단위 캐릭터로 임베딩

# 영문만 다룰 것으로 256개의 글자를 표현
vocab_size = 256 # 총 아스키 코드 수
# 아스키 코드 배열로 정의 후 파이토치 텐서로 변환
x_ = list(map(ord, 'hola')) # ord 함수는 chr 함수와 반대, 캐릭터를 아스키 코드로 변환
y_ = list(map(ord, 'hello'))
x = torch.LongTensor(x_)
y = torch.LongTensor(y_)

In [66]:
# 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.n_layers = 1
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.encoder = nn.GRU(hidden_size, hidden_size)
        self.decoder = nn.GRU(hidden_size, hidden_size)
        
        # 디코더가 번역문의 다음 토큰을 예상해내는 작은 신경망을 하나 더 만들어준다.
        self.project = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs, targets):
        initial_state = self._init_state()
        embedding = self.embedding(inputs).unsqueeze(1)
        
        # 원문을 인코더에 입력시켜 문맥 벡터인 encoder_state를 만들어 낸다
        # 그리고 이 벡터를 디코더의 첫 번째 은닉 벡터 decoder_state로 지정
        # 디코더가 번역문의 첫 번째 토큰을 예상하려면 인코더의 문맥 벡터와 문장 시작 토큰을 입력 데이터로 받아야함
        encoder_output, encoder_state = self.encoder(embedding, initial_state)
        decoder_state = encoder_state
        decoder_input = torch.LongTensor([0]) # 디코더의 문장 시작을 알림
        
        # 디코더는 문장 시작 토큰인 아스키번호 0 을 이용해 번역문 'hola' 의 'h' 토큰을 예측
        # 다음 반복에선 'h' 토큰을 이용해 'o' 토큰을 예측
        # 반복문으로 구현하여 번역 토큰을 순서대로 예상하고 저장
        outputs = []
        for i in range(targets.size()[0]):
            # 디코더는 첫 번째 토큰과 인코더의 문맥 벡터를 동시에 입력 받음
            decoder_input = self.embedding(decoder_input).unsqueeze(1)
            decoder_output, decoder_state = self.decoder(decoder_input, decoder_state)
            
            # 디코더 결과 값은 다시 디코더 모델에 입력
            # 디코더 출력 값이 신경망의 마지막 층인 Softmax 층을 거치면 번역문의 다음 예상 글자가 나옴
            # 이 예상 결과를 outputs 텐서에 저장해 오차를 계산할 때 사용
            # 디코더의 출력값으로 다음 글자 예측
            projection = self.project(decoder_output)
            outputs.append(projection)
            
            # 풍부한 데이터를 바탕으로 학습하는 모델이면 디코더가 예측한 토큰을 
            # 다음 반복에서 입력될 토큰으로 갱신해 주는것이 정석
            # 학습이 부족하면 잘못된 예측 토큰이 입력으로 사용될 가능성이 높다
            # 이를 방지하는 방법 중 티처 포싱(teacher forcing)이 가장 대표적
            # 실제 번역문의 토큰을 디코더의 전 출력값 대신 입력으로 사용해 학습을 가속
            
            # 디처 포싱을 이용한 디코더 입력 갱신
            decoder_input = torch.LongTensor([targets[i]])
            
        outputs = torch.stack(outputs).squeeze()
        return outputs
    
    def _init_state(self, batch_size=1):
        # parameters() 함수는 신경망 모듈 의 가중치 정보들을 반복자 형태로 반환
        # next(self.parameters()).data는 nn.GRU 모듈의 첫번째 가중치 텐서를 추출
        # 그후 new() 함수를 호출해 모델의 가중치와 같은 모양인 (n_layers, batch_size, hidden_dim) 모양을 
        # 갖춘 텐서로 변환후 zero_() 함수를 호출해 텐서 내 모든 값을 0으로 초기화
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

In [67]:
# 학습 시작 전 교차엔트로피 오차를 구하는 CrossEntropyLoss 클래스와 최적화 알고리즘 정의
seq2seq = Seq2Seq(vocab_size, 16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=1e-3)

log = []
for i in range(1000):
    prediction = seq2seq(x, y)
    loss = criterion(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_val = loss.data
    log.append(loss_val)
    if i % 100 == 0:
        print('\n 반복:%d 오차: %s' %(i, loss_val.item()))
        _, top1 = prediction.data.topk(1, 1)
        print([chr(c) for c in top1.squeeze().numpy().tolist()])


 반복:0 오차: 5.4319329261779785
['y', '+', '½', '\x95', '\x95']

 반복:100 오차: 1.9381275177001953
['l', 'l', 'l', 'l', 'l']

 반복:200 오차: 1.0298659801483154
['h', 'l', 'l', 'l', 'l']

 반복:300 오차: 0.697368323802948
['h', 'e', 'l', 'l', 'o']

 반복:400 오차: 0.4219569265842438
['h', 'e', 'l', 'l', 'o']

 반복:500 오차: 0.23388533294200897
['h', 'e', 'l', 'l', 'o']

 반복:600 오차: 0.1421622931957245
['h', 'e', 'l', 'l', 'o']

 반복:700 오차: 0.09422233700752258
['h', 'e', 'l', 'l', 'o']

 반복:800 오차: 0.06706565618515015
['h', 'e', 'l', 'l', 'o']

 반복:900 오차: 0.049924395978450775
['h', 'e', 'l', 'l', 'o']
